In [351]:
import numpy as np
import math
import random
from enum import Enum

In [289]:
uops = {np.sin, np.cos}
bops = {np.add, np.subtract, np.divide, np.multiply, np.power}

# configuration
MIN_OP_LEN = 2
MAX_OP_LEN = 6

MIN_INT_RANGE = 1
MAX_INT_RANGE = 10

is_operation_factor = 0.5


In [352]:
class NodeType(Enum):
    CONST = 1
    OP = 2
    VAR = 3
    EMPTY = 4


class OpType(Enum):
    U = "u"
    B = "b"


class Operator:
    str_mapper = {
        "sin": np.sin,
        "cos": np.cos,
        "add": np.add,
        "sub": np.subtract,
        "divide": np.divide,
        "multiply": np.multiply,
        "power": np.power
    }

    op_mapper = {
        np.sin.__name__: "sin",
        np.cos.__name__: "cos",
        np.add.__name__: "+",
        np.subtract.__name__: "-",
        np.divide.__name__: "/",
        np.multiply.__name__: "*",
        np.power.__name__: "^"
    }
    
    def __init__(self, name):
        if type(name) == str:
            self.op = Operator.str_mapper.get(name, None)
            if self.op is not None:
                if self.op in uops:
                    self.type = OpType.U
                elif self.op in bops:
                    self.type = OpType.B
                else:
                    raise ValueError(f"{self.op} is in str_mapper but not in operation sets")
            else:
                raise ValueError(f"{name} is not a valid operator")

        elif name in uops:
            self.op = name
            self.type = OpType.U
            
        elif name in bops:
            self.op = name
            self.type = OpType.B
            
        else:
            raise ValueError(f"invalid name {name}")

    def __repr__(self):
        return f"Operator[op: {self.op}, type: {self.type}]"

    @staticmethod
    def generate_random_operator(pool: list = None):
        if pool is None:
            pool = pool = list(bops) * 2 + list(uops)
            
        return Operator(random.choice(pool))


class Node:
    
    def __init__(self, value=None, parent: "Node"=None, left: "Node"=None, right: "Node"=None, op=None):
        self.type: NodeType = None
        self.value = value
        self.op = op
        self.parent: "Node" = parent
        self.right: "Node" = right
        self.left: "Node" = left
        
    def add_node(self, node: "Node"):
        if self.left is None:
            self.left = Node
        elif self.right is None:
            self.right = Node
        else:
            raise ValueError("the node doesn't have enough space")

    @property
    def value(self):
        return self._value

    @value.setter
    def value(self, new_value):
        if new_value is None:
            self._value = None
            
        elif new_value == NodeType.EMPTY:
            self._value = None
            self.type = NodeType.EMPTY
            
        elif isinstance(new_value, int):
            self._value = new_value
            self.type = NodeType.CONST
        
        elif isinstance(new_value, str):
            self._value = new_value
            self.type = NodeType.CONST

        else:
            raise ValueError(f"invalid value {new_value}")
    
    @property
    def op(self):
        return self._op

    @op.setter
    def op(self, func):
        if func is None:
            self._op = None
            
        elif not isinstance(func, Operator):
            raise ValueError(f"{func} must be of type {Operator.__name__}")
            
        self._op = func    
        self.type = NodeType.OP

    @staticmethod
    def create_empty_node():
        node = Node()
        node.value = NodeType.EMPTY
        return node
    
    @staticmethod
    def create_random_node():
        return Node(op=random.choice(list(uops + bops)))
    
    def __repr__(self):
        if self.type is not None:
            if self.type == NodeType.OP:
                return f"Node(type: {self.type}, op: {self.op})"
            else:
                return f"Node(type: {self.type}, value: {self.value})"
        else:
            return "Node(None)"
    

class Tree:

    def __init__(self, root=None):
        self.root = root
        self.fitness = 0


    def remove_node(self, node: Node):
        explored: set[Node] = set()
        frontier: list[Node] = [self.root]
        while len(frontier):
            to_explore = frontier.pop()
            if to_explore == node:
                parent = node.parent
                if parent.left == node:
                    parent.left = None
                elif parent.right == node:
                    parent.right = None
            else:
                explored.add(to_explore)
                if to_explore.left not in explored:
                    frontier.insert(0, to_explore.left)
                if to_explore.right not in explored:
                    frontier.insert(0, to_explore.right)

    def _tree_structure(self, node):
        # DFS traversal
        repr_str = ""
        frontier = [node]
        while len(frontier):
            node = frontier.pop()
            if node is not None:
                if node.type == NodeType.OP:
                    repr_str += f"Node(op={node.op}\n"    
                else:
                    repr_str += f"Node(value={node.value})\n"
                
                repr_str += f"L: {node.left}\nR: {node.right}\n---------------\n"

                frontier.append(node.right)
                frontier.append(node.left)

        return repr_str
        
    def __repr__(self):
        return f'Tree(root={self.root}, fitness={self.fitness}, structure:\n{self._tree_structure(self.root)})'

    def __trace(self, node):
        if node is None:
            return ""
        if node.type == NodeType.CONST:
            return str(node.value)
        elif node.type == NodeType.OP:
            sign = Operator.op_mapper.get(node.op.op.__name__)
                
            if node.op.type == OpType.U:
                return f"{sign}({self.__trace(node.left)})"
            elif node.op.type == OpType.B:
                return f"({self.__trace(node.left)} {sign} {self.__trace(node.right)})"
        elif node.type == NodeType.VAR:
            return f"{node.value}"
            
        elif node.type == NodeType.EMPTY:
            return ""

    def __str__(self):
        return self.__trace(self.root)


In [331]:
def random_leaf():
    if random.random() > 0.5:
        new_node = Node()
        new_node.value = random.randint(MIN_INT_RANGE, MAX_INT_RANGE)
    else:
        new_node = Node()
        new_node.value = "x"

    return new_node

In [332]:
def create_random_tree():
    random_op = Operator.generate_random_operator()
    root_node = Node(op=random_op)
    tree = Tree(root_node)
    number_of_operators = random.randint(MIN_OP_LEN, MAX_OP_LEN)
    connected = []
    frontier = [tree.root]
    free = [Node() for _ in range(number_of_operators-1)]
    while len(free) > 0:
        random.shuffle(frontier)
        node = frontier.pop()
        if node.op.type == OpType.B:
            if random.random() > is_operation_factor and len(free) > 1:
                # both children are operators
                ch1 = free.pop()
                ch2 = free.pop()
                ch1.op = Operator.generate_random_operator()
                ch2.op = Operator.generate_random_operator()
                ch1.parent = node
                ch2.parent = node
                
                if random.random() > 0.5:
                    node.left = ch1
                    node.right = ch2
                else:
                    node.left = ch2
                    node.right = ch1
    
                connected.append(node)
                frontier.append(ch1)
                frontier.append(ch2)
                
            else:
                # only one child is operator
                ch = free.pop()
                ch.op = Operator.generate_random_operator()
                ch.parent = node
                if random.random() > 0.5:
                    node.right = ch
                else:
                    node.left = ch
    
                connected.append(node)
                frontier.append(ch)
        
        elif node.op.type == OpType.U:
            ch = free.pop()
            ch.op = Operator.generate_random_operator()
            ch.parent = node
            node.left = ch
            empty_node = Node()
            empty_node.value = NodeType.EMPTY
            connected.append(node)
            frontier.append(ch)
    
    for node in frontier + connected:
        if node.left is None:
            new_node = random_leaf()
            node.left = new_node
            new_node.parent = node

        if node.right is None:
            new_node = random_leaf()
            
            node.right = new_node
            new_node.parent = node
    
    return tree
                

In [349]:
tree = create_random_tree()

In [350]:
print(tree)

(10 + (10 + ((x * (x ^ 5)) - x)))
